In [2]:
from bs4 import BeautifulSoup
import requests

import pandas as pd
import json

from time import sleep
from datetime import datetime, timezone

import boto3

import plotly.io as pio
pio.renderers.default = "iframe_connected"
import plotly.express as px

In [3]:
from sqlalchemy import create_engine, text
import psycopg2

dbuser = ''
dbpass = ''
dbhost = 'first-db.cmgwhx3fnhdo.us-east-1.rds.amazonaws.com'
dbname = 'postgres'

engine = create_engine(f"postgresql+psycopg2://{dbuser}:{dbpass}@{dbhost}/{dbname}", echo=True)

In [4]:
cities_list = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", 
               "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", 
               "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", 
               "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", 
               "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

path = './cities.csv'

# 2) Get weather data from each destination  

## 2.1) Get gps coordinates from nominatim API 🌍

In [5]:
base_url_geo = "https://nominatim.openstreetmap.org/search?"

def nominatim_geocode(address, format='json', limit=1, **kwargs):
    '''
    This wrapper around nominatim API
    Documentation : https://nominatim.org/release-docs/develop/api/Search/
    '''
    params = {"q":address, "format": format, "limit": limit, **kwargs}
    response = requests.get(base_url_geo, params=params)
    response.raise_for_status() # try except r.raise_for_status() ==  None
    sleep(1)
    return response.json()

🗒 **_raise_for_status_** is used to handle exceptions if the status code is not 200  
🗒 **_time.sleep_** is used to delay code execution for some amount of time. Many requests, fired in rapid succession can, depending on the server in question, quickly take up all of the free connections and effectively become a **DoS Attack**. To allow for breathing space, as well as to make sure we don't negatively impact either the users of the website or the website itself, we'd limit the number of requests sent by delaying each one.

## 2.2) Save and structure retrieved data 📚

In [6]:
columns = ['id', 'name', 'latitude', 'longitude']
data = []

for count, city in enumerate(cities_list):
    response = nominatim_geocode(address = city, country = 'France')
    row =[count, city, response[0]['lat'], response[0]['lon']]
    data.append(row)
    
geo_df = pd.DataFrame(data=data, columns=columns)

In [7]:
geo_df.head(10)

,id,name,latitude,longitude
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658
5,5,Paris,48.8588897,2.3200410217200766
6,6,Amiens,49.8941708,2.2956951
7,7,Lille,50.6365654,3.0635282
8,8,Strasbourg,48.584614,7.7507127
9,9,Chateau du Haut Koenigsbourg,48.249489800000006,7.34429620253195


In [8]:
geo_df.dtypes

id            int64
name         object
latitude     object
longitude    object
dtype: object

In [9]:
# Convert latitude and longitude dtypes for the mapbox later
geo_df[["latitude", "longitude"]] = geo_df[["latitude", "longitude"]].apply(pd.to_numeric)

In [10]:
geo_df.dtypes

id             int64
name          object
latitude     float64
longitude    float64
dtype: object

In [11]:
geo_df.to_sql(
    "city",
    engine,
    if_exists='replace',
    index =False
)

2021-12-06 16:04:12,104 INFO sqlalchemy.engine.base.Engine select version()
2021-12-06 16:04:12,105 INFO sqlalchemy.engine.base.Engine {}
2021-12-06 16:04:12,272 INFO sqlalchemy.engine.base.Engine select current_schema()
2021-12-06 16:04:12,274 INFO sqlalchemy.engine.base.Engine {}
2021-12-06 16:04:12,440 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-12-06 16:04:12,444 INFO sqlalchemy.engine.base.Engine {}
2021-12-06 16:04:12,527 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-12-06 16:04:12,528 INFO sqlalchemy.engine.base.Engine {}
2021-12-06 16:04:12,612 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2021-12-06 16:04:12,613 INFO sqlalchemy.engine.base.Engine {}
2021-12-06 16:04:12,779 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

In [13]:
conn = engine.connect()
stmt = text("ALTER TABLE city ADD PRIMARY KEY (id)")
result = conn.execute(stmt)

2021-12-06 16:05:33,286 INFO sqlalchemy.engine.base.Engine ALTER TABLE city ADD PRIMARY KEY (id)
2021-12-06 16:05:33,288 INFO sqlalchemy.engine.base.Engine {}
2021-12-06 16:05:33,459 INFO sqlalchemy.engine.base.Engine COMMIT


## 2.3) Get weather data from One Call API ⛅

In [14]:
base_url_weather = 'https://api.openweathermap.org/data/2.5//onecall?'

def OneCall_weather(lat, lon, exclude, API_key = 'key', units ='metric'):
    '''
    API: One Call
    weather data params: (la,  lon, exclude, api key) 
    url : https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={part}&appid={API key}&units={units}
    format : json (default)
    Documentation : https://openweathermap.org/api/one-call-api
    '''
    params= {'lat': lat, 'lon': lon, 'exclude': exclude, 'APPID':API_key, 'units': units}
    response = requests.get(base_url_weather, params=params)
    response.raise_for_status()
    sleep(1)
    return response.json()

In [15]:
def convertDt(unixDt):
    utc_time = datetime.fromtimestamp(unixDt, timezone.utc)
    local_time = utc_time.astimezone()
    
    return (local_time.strftime("%Y-%m-%d %H:%M:%S (%Z)"))

In [27]:
columns = ['day_time', 'temperature', 'precipitation_p', 'humidity', 'weather', 'cid']
weather_desc =[]
for i in range(len(geo_df)):
    
    cid = geo_df.loc[i, 'id'] # will be used as foreign key
    latitude = geo_df.loc[i, 'latitude']
    longitude = geo_df.loc[i, 'longitude']
    
    response_weather = OneCall_weather(lat = latitude, lon = longitude, exclude = 'current,minutely,hourly,alerts')

    for j in range(1, 8, 1):

        day_time = convertDt(int(response_weather['daily'][j]['dt']))
        
        temperature = response_weather['daily'][j]['temp']['day']
        precipitation_p = response_weather['daily'][j]['pop']
        humidity = response_weather['daily'][j]['humidity']
        weather = response_weather['daily'][j]['weather'][0]['description']
        
        weather_desc.append([day_time, temperature, precipitation_p, humidity, weather, cid])
    
weather_df = pd.DataFrame(weather_desc, columns =columns)

In [29]:
weather_df.head(20)

,day_time,temperature,precipitation_p,humidity,weather,cid
0,2021-12-07 11:00:00 (UTC),8.91,1.00,86,moderate rain,0
1,2021-12-08 11:00:00 (UTC),6.10,0.71,78,moderate rain,0
2,2021-12-09 11:00:00 (UTC),8.87,0.59,75,light rain,0
3,2021-12-10 11:00:00 (UTC),8.35,1.00,62,light rain,0
4,2021-12-11 11:00:00 (UTC),8.27,0.88,67,light rain,0
5,2021-12-12 11:00:00 (UTC),7.38,0.02,84,overcast clouds,0
6,2021-12-13 12:00:00 (UTC),8.25,0.80,94,light rain,0
7,2021-12-07 11:00:00 (UTC),9.61,1.00,87,moderate rain,1
8,2021-12-08 12:00:00 (UTC),6.99,0.73,76,light rain,1
9,2021-12-09 12:00:00 (UTC),9.39,0.56,71,light rain,1


In [30]:
weather_df.to_sql(
    "weather",
    engine,
    if_exists='replace',
)

2021-12-06 16:36:29,481 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2021-12-06 16:36:29,483 INFO sqlalchemy.engine.base.Engine {'name': 'weather'}
2021-12-06 16:36:29,733 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE weather (
	index BIGINT, 
	day_time TEXT, 
	temperature FLOAT(53), 
	precipitation_p FLOAT(53), 
	humidity BIGINT, 
	weather TEXT, 
	cid BIGINT
)


2021-12-06 16:36:29,734 INFO sqlalchemy.engine.base.Engine {}
2021-12-06 16:36:29,907 INFO sqlalchemy.engine.base.Engine COMMIT
2021-12-06 16:36:29,991 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_weather_index ON weather (index)
2021-12-06 16:36:29,993 INFO sqlalchemy.engine.base.Engine {}
2021-12-06 16:36:30,161 INFO sqlalchemy.engine.base.Engine COMMIT
2021-12-06 16:36:30,247 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-12-06 16:36:30,250 INFO sqlalchemy.engine.base.Engine IN

In [31]:
conn = engine.connect()
stmt = text("ALTER TABLE weather ADD PRIMARY KEY (index)")
result = conn.execute(stmt)

2021-12-06 16:37:13,634 INFO sqlalchemy.engine.base.Engine ALTER TABLE weather ADD PRIMARY KEY (index)
2021-12-06 16:37:13,636 INFO sqlalchemy.engine.base.Engine {}
2021-12-06 16:37:13,805 INFO sqlalchemy.engine.base.Engine COMMIT


In [33]:
stmt = text("ALTER TABLE weather ADD CONSTRAINT fk_city FOREIGN KEY (cid) REFERENCES city (id)")
result = conn.execute(stmt)

2021-12-06 16:43:28,417 INFO sqlalchemy.engine.base.Engine ALTER TABLE weather ADD CONSTRAINT fk_city FOREIGN KEY (cid) REFERENCES city (id)
2021-12-06 16:43:28,419 INFO sqlalchemy.engine.base.Engine {}
2021-12-06 16:43:28,635 INFO sqlalchemy.engine.base.Engine COMMIT


## 2.4) Cities where the weather will be the nicest ☀️ 😎

In [69]:
conn = engine.connect()

query = 'SELECT weather.cid, city.id, city.name, city.latitude, city.longitude, AVG(weather.temperature), AVG(weather.precipitation_p) \
         FROM weather \
         join city\
         on city.id=weather.cid\
         GROUP BY  weather.cid, city.id \
         ORDER BY AVG(weather.temperature) desc'

stmt = text(query)
result = conn.execute(stmt)
df_db = pd.DataFrame(result.fetchall(), columns=['cid', 'id', 'name', 'lat', 'lon','avg_temp','avg_prec'])

2021-12-06 17:51:39,795 INFO sqlalchemy.engine.base.Engine SELECT weather.cid, city.id, city.name, city.latitude, city.longitude, AVG(weather.temperature), AVG(weather.precipitation_p)          FROM weather          join city         on city.id=weather.cid         GROUP BY  weather.cid, city.id          ORDER BY AVG(weather.temperature) desc
2021-12-06 17:51:39,798 INFO sqlalchemy.engine.base.Engine {}


In [70]:
df_db.head()

,cid,id,name,lat,lon,avg_temp,avg_prec
0,27,27,Collioure,42.525050,3.083155,10.957143,0.411429
1,33,33,Bayonne,43.493338,-1.475099,9.625714,0.851429
2,32,32,Biarritz,43.471144,-1.552727,9.424286,0.860000
3,25,25,Aigues Mortes,43.565822,4.191284,9.070000,0.368571
4,26,26,Saintes Maries de la mer,43.452277,4.428717,9.052857,0.345714


In [64]:
# mean and sd
weather_means = weather_df.groupby(['cid']).mean()
weather_means = weather_means.sort_values(['temperature','precipitation_p', 'humidity'], ascending = (False, True, True))
weather_means.round(1).head()

,temperature,precipitation_p,humidity
cid,,,
27,11.0,0.4,57.6
33,9.6,0.9,84.1
32,9.4,0.9,83.7
25,9.1,0.4,58.0
26,9.1,0.3,60.6


🗒 Comparing two cities with reference to the three criteria (temperature, humidity and percipitation_p) depends on one's perception of what could be a good weather. It depends on one's life style and especially its region climate. 
🗒  We could have used the Universal Thermal Climate Index (UTCI), however, the One Call API doesn't supply such data.
🗒  According to this article (Weather perception and its impact on out-of-home leisure activity participation decisions: https://www.tandfonline.com/doi/full/10.1080/21680566.2020.1733703), temperature, percipitation and UTCI are the most important factors that may influence someone's decision of going out. That's why, we will sort our data giving three available criteria in this order :  temperature, percipitation, humidity 

In [14]:
def select_best(n, list_):
    # select the first n cities 

    for i in range(n):
        list_.append(weather_means.index[i][0:3])
    return list_

n_best =5
cities_best = pd.DataFrame(select_best(n_best,[]), columns=['City', 'lat', 'lon'])

temp_list =[]
prec_list =[]

for i in cities_best.index:
    (city,lat,lon) = cities_best.loc[i]
    temp_list.append(weather_means.loc[(city,lat,lon),'temperature'])
    prec_list.append(weather_means.loc[(city,lat,lon),'precipitation_p'])
    
cities_best['temperature'] = temp_list
cities_best['precipitation'] = prec_list
cities_best    

,City,lat,lon,temperature,precipitation
0,Collioure,42.525050,3.083155,10.460000,0.457143
1,Bayonne,43.493338,-1.475099,9.512857,0.911429
2,Biarritz,43.471144,-1.552727,9.314286,0.888571
3,Bormes les Mimosas,43.157217,6.329254,8.868571,0.505714
4,Saintes Maries de la mer,43.452277,4.428717,8.845714,0.642857


## 2.5) Export to a csv file 📁

In [88]:
def read_csv(path):
    return pd.read_csv(path, index_col =0)  # index_col =False

def write_csv(data, path):
    data.to_csv(path)

In [16]:
# boto3
session = boto3.Session(aws_access_key_id="key", 
                        aws_secret_access_key="key")

s3 = session.resource("s3")

bucket = s3.create_bucket(Bucket="jedha-project-3")

csv = weather_df.to_csv(index=False)

put_object = bucket.put_object(Key="cities_weather.csv", Body=csv)

## 2.6) Visualize results with mapBox 📊

In [71]:
# read csv file or use dataframe 
df_db.head()

,cid,id,name,lat,lon,avg_temp,avg_prec
0,27,27,Collioure,42.525050,3.083155,10.957143,0.411429
1,33,33,Bayonne,43.493338,-1.475099,9.625714,0.851429
2,32,32,Biarritz,43.471144,-1.552727,9.424286,0.860000
3,25,25,Aigues Mortes,43.565822,4.191284,9.070000,0.368571
4,26,26,Saintes Maries de la mer,43.452277,4.428717,9.052857,0.345714


In [161]:
fig = px.scatter_mapbox(df_db, lat="lat", lon="lon", hover_name='name', color="avg_temp", size ="avg_prec", 
                        hover_data=['avg_temp','avg_prec'], color_discrete_sequence=["bluered"], zoom=5, height=300)

fig.update_layout(mapbox_style="open-street-map")

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [162]:
path2 = './df_each_hotel.csv'
df_hotel = read_csv(path2)

In [163]:
df_hotel.sort_values(by =['city_id', 'hotel_score'], ascending =(True, False))
df_db
n_city = 5
n_hotel = 5
df_f = pd.DataFrame()
for i in range(n_city):
    city_id = df_db.loc[i,'cid']
    best_hot = df_hotel.loc[df_hotel['city_id'] == city_id].sort_values(by='hotel_score', ascending=False).head(n_hotel)
    df_f = pd.concat([df_f, best_hot], ignore_index=True)

In [164]:
df_f.head()

,city_id,city_name,hotel_id,hotel_name,hotel_url,hotel_score,hotel_text,lattitude,longitude
0,27,Collioure,8,Le Relais Des Trois Mas,https://www.booking.com//hotel/fr/le-relais-de...,9.0,\nDoté d'une terrasse avec piscine donnant sur...,42.524173,3.089311
1,27,Collioure,5,Le Madeloc Hôtel & Spa,https://www.booking.com//hotel/fr/madeloc.fr.h...,8.8,\nSitué à 500 mètres du centre-ville de Collio...,42.528765,3.078654
2,27,Collioure,9,Hôtel La Casa Pairal,https://www.booking.com//hotel/fr/hotelcasapai...,8.7,\nSitué à 250 mètres de la plage et du port de...,42.526167,3.082356
3,27,Collioure,2,Hôtel la Bona Casa,https://www.booking.com//hotel/fr/la-bona-casa...,8.4,\nL'Hôtel la Bona Casa est situé dans le centr...,42.525472,3.082699
4,27,Collioure,0,Hôtel Princes de Catalogne,https://www.booking.com//hotel/fr/princes-de-c...,8.1,\nSitué sur une place tranquille dans la ville...,42.525914,3.082682


In [165]:
fig = px.scatter_mapbox(df_f, lat="lattitude", lon="longitude", hover_name='hotel_name', color = 'hotel_score', size ="hotel_score", 
                        hover_data=['city_name','hotel_score'], color_continuous_scale= "ylorrd", zoom=5, height=300)

fig.update_layout(mapbox_style="open-street-map")

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [170]:
import plotly.express as px
import plotly.graph_objects as go


fig = px.scatter(px.scatter_mapbox(df_db.head(), lat="lat", lon="lon", hover_name='name', color="avg_temp", size ="avg_prec", 
                        hover_data=['avg_temp','avg_prec'], color_discrete_sequence=["bluered"], zoom=5, height=300))

fig2 = px.scatter_mapbox(df_f, lat="lattitude", lon="longitude", hover_name='hotel_name', color = 'hotel_score', size ="hotel_score", 
                        hover_data=['city_name','hotel_score'], color_continuous_scale= "ylorrd", zoom=5, height=300)

        
fig.add_trace(fig2.data[0])

fig.update_layout(mapbox_style="open-street-map")

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()